# Description 
### Extracting Postal Code, Borough, Neighborhood data of Canada, droping Not Assigned Borough and Fill in Not Assigned Neighborhood data with Borough

     |████████████████████████████████| 122kB 6.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.5MB 5.6MB/s eta 0:00:01     |█████                           | 860kB 5.6MB/s eta 0:00:01     |████████████████████████        | 4.1MB 5.6MB/s eta 0:00:01


In [1]:
!pip install bs4
import pandas as pd 
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize 
import requests as r
from bs4 import BeautifulSoup
!pip install lxml

     |████████████████████████████████| 122kB 6.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.5MB 11.0MB/s eta 0:00:01


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [13]:
source = r.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content
soup=BeautifulSoup(source,'html.parser')
print(soup.title)

tab = str(soup.table)
table = pd.read_html(tab)
df = table[0]
df

<title>List of postal codes of Canada: M - Wikipedia</title>


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [14]:
df = df[df.Borough !="Not assigned"]
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
# Combining the neighbourhoods with same Postalcode
df = df.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)

import numpy as np
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

df.shape

(103, 3)

,,Neighborhood
Postal Code,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


In [41]:
coor = pd.read_csv('Geospatial_Coordinates.csv')
coor

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [37]:
!pip install geocoder

     |████████████████████████████████| 102kB 4.8MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.8MB/s eta 0:00:011
     |████████████████████████████████| 829kB 7.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [34]:
df.drop(['index'],axis=1)
df

,index,Postal Code,Borough,Neighborhood,lat
0,0,M3A,North York,Parkwoods,pina
1,1,M4A,North York,Victoria Village,pina
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",pina
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",pina
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",pina
...,...,...,...,...,...
98,98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",pina
99,99,M4Y,Downtown Toronto,Church and Wellesley,pina
100,100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",pina
101,101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",pina


In [47]:
merged_df = pd.merge(df,coor,
                    left_on = 'Postal Code',
                    right_on = 'Postal Code',
                    how = 'left')
merged_df
merged_df.drop('lat',axis=1,inplace=True)
merged_df
merged_df.drop('index',axis=1,inplace=True)
merged_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
